# 文件读取

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import faiss
import pandas as pd
import numpy as np
import torch
import os
from transformers import AutoTokenizer, AutoModel
from openai import OpenAI
import torch

In [2]:
os.chdir(r"E:\天池比赛")
df = pd.read_csv(r".\A_audio_recognition_dom_189_train_val_avg.txt", sep='\t')
df_query = pd.read_csv(r"E:\天池比赛\微调\df_query_506.csv")
df_query = df_query[['uuid','text','food_feature','feature_class']]


# 相似度分类

In [3]:
path = "./BAAI"
# 初始化模型和分词器
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

# 菜品库-读取时将list转回tensor
import ast
df_food = pd.read_csv(r'./df_food.csv')
df_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x))) 


In [4]:
#转换词向量函数
def cls(text): 
    input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**input).last_hidden_state
    cls_embedding = output[:, 0, :]
    cls_embedding = cls_embedding[0]
    return cls_embedding

In [5]:
df_query['cls'] = df_query['food_feature'].progress_apply(cls)

100%|██████████| 1996/1996 [03:43<00:00,  8.93it/s]


### 菜品库向量FAISS 存储

In [6]:
# 把向量堆叠成 numpy 数组（shape: N x dim）
embeddings = np.stack(df_food['cls'].values).astype('float32')
dim = embeddings.shape[1]

# 创建索引（使用 L2 距离）
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [7]:
def query(food_cls):
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=1)  # D: 距离, I: 索引
    formatted = df_food.loc[I[0][0],'item_name']
    return formatted

# 最终推荐菜品与提取特征间的相似度
def similarity(cls1,cls2):
    similarity = torch.cosine_similarity(cls1, cls2, dim=0)
    return similarity.item()

In [8]:
#查询
df_query['Query_faiss'] = df_query.progress_apply(lambda row: query(row['cls']),axis=1)
df_query['cls1'] = df_query['Query_faiss'].progress_apply(cls)
df_query['similarity'] = df_query.progress_apply(lambda row: similarity(row['cls'], row['cls1']), axis=1)


100%|██████████| 1996/1996 [00:00<00:00, 21443.63it/s]


In [9]:
df_query_1 = df_query[df_query['similarity']>=0.9]
df_query_2 = df_query[(df_query['similarity']>=0.7) & (df_query['similarity'] < 0.9)]
df_query_3 = df_query[df_query['similarity']<=0.7]

In [13]:
df_query_1.to_excel(r'./微调/数据分类/df_query_1.xlsx')
df_query_2.to_excel(r'./微调/数据分类/df_query_2.xlsx')
df_query_3.to_excel(r'./微调/数据分类/df_query_3.xlsx')

In [10]:
#第一部分
df_query_1['recommend'] = df_query_1['Query_faiss']

C:\Users\guofengrun\AppData\Local\Temp\ipykernel_15556\2331399883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_1['recommend'] = df_query_1['Query_faiss']


In [11]:
#第二部分

In [12]:
def query2(food_cls,feature_class):
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=5)  # D: 距离, I: 索引
    formatted = list(df_food.loc[s,'item_name'] for i, s in enumerate(I[0]))
    if feature_class == '0':
        formatted += ['营养配餐粥','养胃小米粥','小份烂糊白菜','骨汤鸡蛋汤','清炒时蔬菜']
    return formatted

In [14]:
#查询
df_query_2['Query_faiss'] = df_query_2.progress_apply(lambda row: query2(row['cls'],row['feature_class']),axis=1)

100%|██████████| 465/465 [00:01<00:00, 245.95it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_15556\1526680946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_2['Query_faiss'] = df_query_2.progress_apply(lambda row: query2(row['cls'],row['feature_class']),axis=1)


In [15]:
## nlp推荐
def nlp_recommend(food_name,food_choice):
    input = food_name+'+'+str(food_choice)
    message = [
        {'role': 'system', 'content': '''你是一个智能点餐助手。给你一个点餐文本，以及可能的菜品列表。你的任务是从菜品列表中选出最符合点餐文本的菜品，并且只返回该菜品名称。
                    例如：
                    user：天猫精灵要个家常花菜炒肉片吧+[花菜炒肉片, 菜花炒肉片, 家常炒花菜, 花菜家常炒, 花菜炒肉丝]
                    assistant:花菜炒肉片'''},
        {'role': 'user', 'content':input}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen2.5-32b-instruct", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [16]:
df_query_2['recommend'] = df_query_2.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)

100%|██████████| 465/465 [08:23<00:00,  1.08s/it]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_15556\3231103119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_2['recommend'] = df_query_2.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)


In [18]:
df_query_2

,uuid,text,food_feature,feature_class,cls,Query_faiss,cls1,similarity,recommend
3,6dc1df30-6d8c-40a5-9be9-52f398623f07,天猫精灵鲜鱼豆腐汤来一份吧,鲜鱼豆腐汤,1,"[tensor(-0.1967), tensor(0.9198), tensor(0.711...","[三鲜豆腐汤, 鲫鱼豆腐汤, 豆腐鲫鱼汤, 水鱼豆腐汤, 海鲜豆腐汤]","[tensor(-0.2713), tensor(0.0347), tensor(0.867...",0.847946,鲫鱼豆腐汤
5,e54d8c37-1c8c-4b76-a7be-1e416e66280b,天猫精灵来碗萝卜面疙瘩热汤吧,萝卜面疙瘩热汤,1,"[tensor(0.3594), tensor(-0.2250), tensor(0.731...","[萝卜疙瘩汤, 萝卜丝疙瘩汤, 白萝卜疙瘩汤, 西红柿面疙瘩汤, 西红柿热面汤]","[tensor(0.5440), tensor(0.3332), tensor(0.6148...",0.832524,萝卜疙瘩汤
12,19351f4a-5c2d-4416-aa7c-d8516bbe8ba9,天猫精灵来碗绿豆清凉粥,绿豆清凉粥,1,"[tensor(0.6221), tensor(-0.1144), tensor(-0.13...","[清爽绿豆粥, 清火绿豆粥, 清香绿豆粥, 清甜绿豆粥, 绿豆解暑粥]","[tensor(1.3211), tensor(-0.5416), tensor(-0.33...",0.896710,清爽绿豆粥
22,c6c78600-7e83-415d-8664-f4e3c0b587e1,天猫精灵来份番茄鸡蛋浇头饭吧,番茄鸡蛋浇头饭,1,"[tensor(0.4777), tensor(-0.1305), tensor(0.506...","[番茄鸡蛋盖浇饭, 番茄鸡蛋饭, 番茄炒鸡蛋盖浇饭, 番茄鸡蛋拌饭, 番茄炒蛋浇头]","[tensor(0.1786), tensor(0.3527), tensor(0.4827...",0.895699,番茄鸡蛋盖浇饭
28,e04b0fbc-6749-48e7-b12c-9bcefd4c2b66,来份鲜鱼豆腐汤吧,鲜鱼豆腐汤,1,"[tensor(-0.1967), tensor(0.9198), tensor(0.711...","[三鲜豆腐汤, 鲫鱼豆腐汤, 豆腐鲫鱼汤, 水鱼豆腐汤, 海鲜豆腐汤]","[tensor(-0.2713), tensor(0.0347), tensor(0.867...",0.847946,鲫鱼豆腐汤
...,...,...,...,...,...,...,...,...,...
1983,dc0517bd-0d75-4011-bb81-9f14e94b71aa,天猫精灵来碗养胃的小米粥吧,小米粥,1,"[tensor(0.8989), tensor(0.8290), tensor(-0.233...","[大米小米粥, 粗粮小米粥, 早餐小米粥, 小米粥南瓜粥, 牛奶小米粥]","[tensor(1.1569), tensor(1.1570), tensor(0.1737...",0.890502,小米粥南瓜粥
1989,47f5bac4-d6d9-418c-80ab-178490ff9612,来份热乎养胃的小米粥吧,小米粥,1,"[tensor(0.8989), tensor(0.8290), tensor(-0.233...","[大米小米粥, 粗粮小米粥, 早餐小米粥, 小米粥南瓜粥, 牛奶小米粥]","[tensor(1.1569), tensor(1.1570), tensor(0.1737...",0.890502,小米粥南瓜粥
1990,615f8c03-6c18-45c5-8969-ebdc58c01c07,天猫精灵来碗暖心的菌菇汤吧,菌菇汤,1,"[tensor(0.5653), tensor(0.2029), tensor(-0.520...","[养生菌菇汤, 砂锅菌菇汤, 野生菌菇汤, 鲜美菌菇汤, 松茸菌菇汤]","[tensor(0.6869), tensor(0.3218), tensor(-0.430...",0.895828,养生菌菇汤
1991,9b2b32da-dba8-447d-b090-84b48acb823d,来碗番茄汤小浑炖面吧,番茄汤小浑炖面,1,"[tensor(0.5655), tensor(0.6166), tensor(0.8255...","[番茄浓汤面, 番茄蛋汤面, 番茄鸡蛋汤面, 浓汤番茄鸡蛋面, 番茄挂面汤]","[tensor(0.2080), tensor(0.2614), tensor(0.7108...",0.834274,番茄浓汤面


## faiss审核确保输出

In [19]:
def query_post(recommend):
#返回最相近的菜品
    food_cls = cls(recommend)
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=1)  # D: 距离, I: 索引
    return df_food.loc[I[0][0],'item_name']

In [20]:
#查询
df_query_2['recommend'] = df_query_2['recommend'].progress_apply(query_post)

100%|██████████| 465/465 [00:56<00:00,  8.25it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_15556\1808123285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query_2['recommend'] = df_query_2['recommend'].progress_apply(query_post)


In [ ]:
#第三部分

In [39]:
def recomend3(keyword_list,food_name):
    mask = df_query_3['food_feature'].str.contains('|'.join(keyword_list))
    df_query_3.loc[mask, 'recommend'] = food_name


In [52]:
recomend3(['少油','少盐','不油腻','少调料','蔬菜','不要太咸','别太咸','少放点调料','不要油','不要太咸','少调味的菜'],'清炒时蔬菜')
recomend3(['养胃','易消化','容易下咽','好消化','顺口','好咽'],'养胃小米粥')
recomend3(['清淡的'],'清淡白米粥')
recomend3(['烂','软','糯','软乎','容易嚼的菜','不费牙','好嚼','温乎的饭菜','易嚼','炖得久的菜'],'小份烂糊白菜')
recomend3(['营养'],'大份营养鸡汤')
recomend3(['汤'],'西红柿紫菜汤')
recomend3(['主食','软乎面食'],'软香大米饭')
recomend3(['家常'],'家常炒鸡不辣')
recomend3(['容易嚼的','容易嚼的肉'],'皮蛋瘦肉粥')
recomend3(['不要辣'],'家常炒鸡不辣')
recomend3(['热乎','温乎','暖和'],'温热南瓜汤')
recomend3(['无特定要求'],'皮蛋瘦肉粥')

#微调更好
recomend3(['红枣香米粥'],'大米红枣粥')
recomend3(['小炒'],'滋补养生小炒')
recomend3(['罗式虾'],'招牌鲜虾粥')
recomend3(['适合老年人的菜'],'营养八宝粥')


In [53]:
#拼接3部分
df_query_0 = pd.concat([df_query_1, df_query_2, df_query_3], axis=0)

# 提交文件格式

In [58]:
df_result = pd.merge(df, df_query_0[['uuid','recommend']], how='left', on='uuid')

In [59]:
df_result.columns

Index(['uuid', 'text', 'dom', 'recommend'], dtype='object')

In [60]:

df_result.to_csv(r".\result\Results507.txt", 
          sep='\t', 
          header=None, 
          index=False, 
          na_rep='')

In [38]:
# 存储时将tensor转换为list
#df_food['cls'] = df_food['cls'].apply(lambda x: x.tolist())
#df_food.to_csv(r'./df_food.csv',index=False)

In [ ]:

'''
# 读取时将list转回tensor
import ast
df_food = pd.read_csv(r'./df_food.csv')
df_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x)))
'''

In [40]:
#df_query[df_query['similarity']<0.7][['text','Query_faiss','nlp_post','similarity']].to_excel('./微调/模糊菜品430.xlsx')

In [41]:
#df_query.to_csv(r'./微调/df_query.csv')